In [ ]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@File         :AutoModel.ipynb
@Description  :用automodel方式完成多种NLP任务
@Time         :2022/04/28 20:40:19
@Author       :Hedwig
@Version      :1.0
'''


In [ ]:
# AutoModel类用于管理Transformers库中处理相同地NLP任务的底层具体模型，为上层应用管道方式提供统一的接口
# 按Bertology系列的应用场景，Transformer库被划分为以下6个子类
# AutoModel：基本载入类，适用于Transformers中的任何模型，也适用于特征提取
# AutoModelForPreTraining：特征提取任务的模型载入类，适用于Transformers库中所有的预训练模型
# AutoModelForSequenceClassification：文本分类的模型载入类，适用于Transformers中所有文本分类模型
# AutoModelForQuestionAnswering：阅读理解任务的模型载入类，适用于Transformers中所有抽取式问答模型
# AutoModelWithLMHead：完形填空任务的模型载入类，适用于Transformers中所有遮蔽语言模型
# AutoModelForTokenClassification：实体词识别的模型载入类，适用于Transformers库中所有实体词识别模型

In [1]:
from transformers import *

In [ ]:
# transformers的models/auto路径下有modeling_auto.py源文件，可以找到模型载入类和具体的模型映射关系
# 以AutoModelWithLMHead类为例，MODEL_WITH_LM_HEAD_MAPPING_NAMES代表了载入类与系列模型的映射
# 在这里列出的所有元素都可以实现AutoModelWithLMHead类所完成的完形填空
# 它的元素包括两部分，具体模型的配置文件和具体模型的实现类
# 每一个具体模型的实现类会通过不同数据集被训练成多套预训练模型文件
# 每套模型训练文件又由3-4个子文件组成：词表文件、词表扩展文件(可选)、配置文件和模型权值文件
# 这些文件共用一个统一的字符串标识
# 用自动加载方式调用模型时，系统会根据统一的预训练模型字符串标识，找到对应的预训练模型文件，通过网络下载并载入 

In [ ]:
# 具体地，pipeline的任务字符串(以fill-mask为例)